In [ ]:
# import logging
# from telegram import Update
# from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext, ApplicationBuilder
# import requests
# import json

# # Enable logging
# logging.basicConfig(
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     level=logging.INFO
# )

# logger = logging.getLogger(__name__)

# # Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your actual token
# TELEGRAM_TOKEN = '7212830442:AAGldEXk5zN4Ri-fteXKugqIvXJbJ0149k0'
# OLLAMA_API_URL = "http://localhost:11434/api/generate"

# # Function to generate response from Ollama
# # def generate_response(input_text):
# #     headers = {"Content-Type": "application/json"}
# #     payload = {
# #         "prompt": input_text,
# #         "model": "llama3",
# #         "options": {"stream": False}  # Set to False for synchronous response
# #     }
# #     try:
# #         response = requests.post(OLLAMA_API_URL, headers=headers, data=json.dumps(payload))
# #         response.raise_for_status()

# #         # Process line-by-line JSON objects
# #         responses = []
# #         for line in response.text.splitlines():
# #             try:
# #                 data = json.loads(line)
# #                 if 'response' in data:
# #                     responses.append(data['response'])
# #                 if data.get("done", False):
# #                     break
# #             except json.JSONDecodeError as e:
# #                 logger.error(f"JSON decode error: {e}. Raw line: {line}")

# #         # Join all the pieces together
# #         return ' '.join(responses) if responses else "I am not sure how to respond."

# #     except requests.exceptions.RequestException as e:
# #         logger.error(f"API request failed: {e}")
# #         return "I am unable to process your request at the moment. Please try again later."
# def generate_response(input_text):
#     headers = {"Content-Type": "application/json"}
#     payload = {
#         "prompt": input_text,
#         "model": "llama3",
#         "options": {"stream": True}  # Stream for word-by-word response
#     }
#     try:
#         response = requests.post(OLLAMA_API_URL, headers=headers, data=json.dumps(payload))
#         response.raise_for_status()

#         responses = []
#         for line in response.text.splitlines():
#             try:
#                 data = json.loads(line)
#                 if 'response' in data:
#                     responses.append(data['response'])
#                 if data.get("done", False):
#                     break
#             except json.JSONDecodeError as e:
#                 logger.error(f"JSON decode error: {e}. Raw line: {line}")

#         return ' '.join(responses) if responses else "I am not sure how to respond."

#     except requests.exceptions.RequestException as e:
#         logger.error(f"API request failed: {e}")
#         return "I am unable to process your request at the moment. Please try again later."


# # Define command handlers
# async def start(update: Update, context: CallbackContext):
#     """Send a welcome message when the /start command is issued."""
#     await update.message.reply_text('Hello! I am your AI assistant. How can I help you today?')

# async def help_command(update: Update, context: CallbackContext):
#     """Send a message when the /help command is issued."""
#     await update.message.reply_text('You can ask me anything by typing your message.')

# async def handle_message(update: Update, context: CallbackContext):
#     """Handle incoming messages from the user."""
#     user_message = update.message.text
#     user_id = update.message.chat_id
#     logger.info(f"User ({user_id}) says: {user_message}")

#     # Generate response from Ollama
#     try:
#         response_text = generate_response(user_message)
#         await update.message.reply_text(response_text)
#     except Exception as e:
#         logger.error(f"Error generating response: {e}")
#         await update.message.reply_text("Sorry, I encountered an error while processing your request.")

# async def error(update: Update, context: CallbackContext):
#     """Log errors caused by Updates."""
#     logger.error(f"Update {update} caused error {context.error}")

# def main():
#     """Start the bot."""
#     application = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

#     # Register command handlers
#     application.add_handler(CommandHandler("start", start))
#     application.add_handler(CommandHandler("help", help_command))

#     # Register a message handler
#     application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

#     # Log all errors
#     application.add_error_handler(error)

#     # Start the Bot
#     application.run_polling()

#     logger.info("Bot started. Listening for messages...")

# if __name__ == '__main__':
#     main()

import logging
import os
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
import requests
import json

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger(__name__)

# Fetch Telegram token from environment variable
TELEGRAM_TOKEN = '7212830442:AAGldEXk5zN4Ri-fteXKugqIvXJbJ0149k0'
if not TELEGRAM_TOKEN:
    logger.error("TELEGRAM_BOT_TOKEN not found in environment variables.")
    exit(1)

OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Function to generate response from Ollama word by word
def generate_response(input_text):
    headers = {"Content-Type": "application/json"}
    payload = {
        "prompt": input_text,
        "model": "deepseek-r1:14b",
        "options": {"stream": True}  # Stream for word-by-word response
    }
    try:
        response = requests.post(OLLAMA_API_URL, headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        responses = []
        for line in response.text.splitlines():
            try:
                data = json.loads(line)
                if 'response' in data:
                    responses.append(data['response'])
                if data.get("done", False):
                    break
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}. Raw line: {line}")

        return ' '.join(responses) if responses else "I am not sure how to respond."

    except requests.exceptions.RequestException as e:
        logger.error(f"API request failed: {e}")
        return "I am unable to process your request at the moment. Please try again later."

# Define command handlers
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text('Hello! I am your AI assistant. How can I help you today?')

async def help_command(update: Update, context: CallbackContext):
    await update.message.reply_text('You can ask me anything by typing your message.')

async def handle_message(update: Update, context: CallbackContext):
    user_message = update.message.text
    user_id = update.message.chat_id
    logger.info(f"User ({user_id}) says: {user_message}")

    # Generate response from Ollama
    response_text = generate_response(user_message)
    await update.message.reply_text(response_text)

async def error(update: Update, context: CallbackContext):
    logger.error(f"Update {update} caused error {context.error}")

# Main function to run the bot
def main():
    application = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    application.add_error_handler(error)

    application.run_polling()
    logger.info("Bot started. Listening for messages...")

if __name__ == '__main__':
    main()

In [ ]:
import logging
import os
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, Filters, CallbackContext
import requests
import json
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from googlesearch import search as google_search

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Telegram token
TELEGRAM_TOKEN = '7212830442:AAGldEXk5zN4Ri-fteXKugqIvXJbJ0149k0'
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Đường dẫn tới thư mục chứa nhiều tài liệu
DOCUMENTS_DIR = "path_to_your_documents_folder"

# Khởi tạo embeddings và vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = None

# Tải và xử lý nhiều tài liệu
def load_and_process_documents():
    global vector_store
    try:
        all_chunks = []
        for filename in os.listdir(DOCUMENTS_DIR):
            if filename.endswith(".pdf"):
                file_path = os.path.join(DOCUMENTS_DIR, filename)
                loader = PyPDFLoader(file_path)
                documents = loader.load()
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
                chunks = text_splitter.split_documents(documents)
                all_chunks.extend(chunks)
        if all_chunks:
            vector_store = FAISS.from_documents(all_chunks, embeddings)
            logger.info("Multiple documents loaded and vectorized successfully.")
        else:
            logger.warning("No documents found in the directory.")
    except Exception as e:
        logger.error(f"Error loading documents: {e}")

# Truy xuất thông tin từ tài liệu
def retrieve_from_document(query):
    if vector_store is None:
        return "Documents not loaded yet."
    try:
        docs = vector_store.similarity_search(query, k=3)
        return "\n".join([doc.page_content for doc in docs])
    except Exception as e:
        logger.error(f"Error retrieving from documents: {e}")
        return "Unable to retrieve information from the documents."

# Tìm kiếm web
def search_web(query):
    try:
        results = list(google_search(query, num_results=3))
        return "\n".join([f"Web result: {url}" for url in results])
    except Exception as e:
        logger.error(f"Web search failed: {e}")
        return "Unable to fetch web results."

# Tìm kiếm trên X (dùng khả năng của Grok)
def search_x(query):
    # Giả lập tìm kiếm X bằng cách tận dụng khả năng của Grok
    # Trong thực tế, tôi sẽ gọi API nội bộ của xAI để tìm kiếm X
    return f"X search results for '{query}' (simulated by Grok)."

# Tối ưu hóa prompt
def optimize_prompt(query, retrieved_info, external_info):
    return f"""
    You are Grok, created by xAI. Answer the following query concisely and accurately, using the provided information if relevant. Avoid unnecessary filler and focus on delivering value.

    User query: {query}
    Retrieved from documents: {retrieved_info}
    External info (web/X): {external_info}
    Provide a clear, concise, and helpful response:
    """

# Gửi yêu cầu tới Ollama với prompt tối ưu
def generate_response_with_rag(query):
    # Truy xuất từ tài liệu
    retrieved_info = retrieve_from_document(query)
    
    # Tìm kiếm bên ngoài nếu từ khóa "search" hoặc "web" hoặc "X" xuất hiện
    external_info = ""
    if "search" in query.lower() or "web" in query.lower():
        external_info = search_web(query)
    elif "x" in query.lower():
        external_info = search_x(query)

    # Tối ưu hóa prompt
    optimized_prompt = optimize_prompt(query, retrieved_info, external_info)
    
    headers = {"Content-Type": "application/json"}
    payload = {
        "prompt": optimized_prompt,
        "model": "deepseek-r1:14b",
        "options": {"stream": True}
    }
    try:
        response = requests.post(OLLAMA_API_URL, headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        responses = []
        for line in response.text.splitlines():
            try:
                data = json.loads(line)
                if 'response' in data:
                    responses.append(data['response'])
                if data.get("done", False):
                    break
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}. Raw line: {line}")

        return ' '.join(responses) if responses else "I am not sure how to respond."
    except requests.exceptions.RequestException as e:
        logger.error(f"API request failed: {e}")
        return "I am unable to process your request at the moment. Please try again later."

# Command handlers
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text('Hello! I am your RAG-powered AI assistant. I can search documents, web, or X. Ask me anything!')

async def help_command(update: Update, context: CallbackContext):
    await update.message.reply_text('Ask me anything. Use "search", "web", or "X" to trigger external searches.')

async def handle_message(update: Update, context: CallbackContext):
    user_message = update.message.text
    user_id = update.message.chat_id
    logger.info(f"User ({user_id}) says: {user_message}")

    response_text = generate_response_with_rag(user_message)
    await update.message.reply_text(response_text)

async def error(update: Update, context: CallbackContext):
    logger.error(f"Update {update} caused error {context.error}")

# Main function
def main():
    # Tải tài liệu khi khởi động
    load_and_process_documents()

    # Khởi động bot Telegram
    application = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

    application.add_error_handler(error)

    application.run_polling()
    logger.info("Bot started. Listening for messages...")

if __name__ == '__main__':
    main()